# 전국 스타벅스 매장 정보 가져오기

# 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook
from selenium import webdriver
# 웹 화면이 로딩될 때까지 기다려주는 모듈
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# HTML 파일 읽어오기
html 파일 읽어오기(requests라이브러리, BeautifulSoup라이브러리)

In [15]:
# 웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("headless")

driver = webdriver.Chrome("**********************************", options=chrome_options)
# 페이지 전체가 로딩되는 시간까지 10초
driver.implicitly_wait(10)

# 스타벅스 매장 지도 url
starbucks_map_url = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver.get(starbucks_map_url)

/var/folders/3t/q5j26dq54hjgz7mhrg40ymbw0000gp/T/ipykernel_27334/2286036863.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/jh931201/AI6-likelion/chromedriver", options=chrome_options)


In [4]:
# 스타벅스 서울
starbucks_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_css))).click()

## 스타벅스 서울 매장 위치 HTML 읽어오기

In [5]:
# 스타벅스 서울 전체 클릭
starbucks_seoul_all_css = "#mCSB_2_container > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all_css))).click()

In [6]:
# 스타벅스 서울 전체 HTML 파싱
html = driver.page_source
soup = bs(html, "lxml")

In [7]:
li_all = soup.select("#mCSB_3_container > ul > li")
li_all[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [8]:
starbucks_data = []

for li in li_all:
    name = li.find("strong").text.strip()
    address = li.find("p").text.strip().replace("1522-3232", "")
    city = address.split(" ")[0]
    gu = address.split(" ")[1]
    load = address.split(" ", 2)[2]
#     print(name, address, gu, sep=" | ")
    each = {
        "매장이름": name,
        "시": city,
        "구": gu,
        "도로명주소": load
    }
    starbucks_data.append(each)

In [9]:
len(starbucks_data)

578

In [10]:
df_starbucks = pd.DataFrame(starbucks_data)
df_starbucks.tail()

,매장이름,시,구,도로명주소
573,중랑구청,서울특별시,중랑구,신내로 72
574,사가정역,서울특별시,중랑구,면목로 310
575,상봉역,서울특별시,중랑구,망우로 307 (상봉동)
576,묵동,서울특별시,중랑구,"동일로 952 (묵동, 로프트원 태릉입구역) 1층"
577,중화역,서울특별시,중랑구,"봉화산로 35 1,2층"


In [11]:
df_starbucks.describe()

,매장이름,시,구,도로명주소
count,578,578,578,578
unique,578,1,25,573
top,역삼아레나빌딩,서울특별시,강남구,경인로 662 (신도림동)
freq,1,578,88,2


In [12]:
df_starbucks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장이름    578 non-null    object
 1   시       578 non-null    object
 2   구       578 non-null    object
 3   도로명주소   578 non-null    object
dtypes: object(4)
memory usage: 18.2+ KB


# 파일로 저장하기

In [13]:
# 저장할 파일명
file_name = "starbucks_seoul_local.csv"
# CSV 파일로 저장하기
df_starbucks.to_csv(file_name, index=False)
# 저장한 파일 확인
pd.read_csv(file_name)

,매장이름,시,구,도로명주소
0,역삼아레나빌딩,서울특별시,강남구,언주로 425 (역삼동)
1,논현역사거리,서울특별시,강남구,강남대로 538 (논현동)
2,신사역성일빌딩,서울특별시,강남구,강남대로 584 (논현동)
3,국기원사거리,서울특별시,강남구,테헤란로 125 (역삼동)
4,대치재경빌딩R,서울특별시,강남구,남부순환로 2947 (대치동)
...,...,...,...,...
573,중랑구청,서울특별시,중랑구,신내로 72
574,사가정역,서울특별시,중랑구,면목로 310
575,상봉역,서울특별시,중랑구,망우로 307 (상봉동)
576,묵동,서울특별시,중랑구,"동일로 952 (묵동, 로프트원 태릉입구역) 1층"
